**Goal:**

Everyday, check the predicted stock price for the next day, and see if the predicted price gets close to upper or lower band of the bollinger bands. If the predicted price gets close to upper band, we sell all the particular stocks at hand; if the predicted price gets close to lower band, we buy the stock the next day.

To simplify things, every time we buy a stock, we allocate half of current cash for it (including the transaction cost).

To avoid low ROI transaction cost, we decide that after day 700(TBD) we will stop buying stocks (since we only have limited profit window in the last 50 days).

Desired output:
A table documenting: on each decision day (prediction day):

For each stock:
0. The start unit at hand
1. The predicted next 1 day price
2. The bollinger upper and lower band calculated by the actual price of the last 50 days
4. The buy/sell price for this stock (the actual next 1 day price)
5. The number of bought units for this stock
6. The number of sold units for this stock
7. The end units at hand

For each day:
1. Decision day
2. Trade day
3. Start cash
4. Transaction cost
5. End cash


**Steps:**
1. Get the prediction function ready
2. Get the actual price dataframe ready
3. Write up the trading logic and limits
4. Try out five companies at first to test run the program
5. Try running the program on all these stock combo: all stocks, selected high variance stock, selected low variance stock, selected portfolio; pick the best combo
6. Remember the decision days are 504-755. On day 756, we need to sell everything we're still holding.

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('max_columns', None)

# Prepare the data

In [21]:
data = pd.read_csv('/Users/rachel/Desktop/Spring/DSO 530 Machine Learning/group project/final final variable/Next_1_Close_y.csv', index_col=0)

raw = pd.read_csv('/Users/rachel/Desktop/Spring/DSO 530 Machine Learning/group project/final final variable/strategy/raw.csv')

data['Company'] = raw['Company']
data['Time'] = raw['Time']
data['True Price'] = raw['Close']

day=[]
for i in range(1,17):
    day.append('Day{}'.format(i))
    
data = data[data['Time'].isin(day)==False]

data = data[["Price_rate5_change", "OBVlag", 'PROClag', "MA5lag", "day15Returnlag", 'Relative_10_LogVolume', 
             'Next_1_Close', 'Company', 'Time', 'True Price']]

meanL = []
highL = []
lowL = []

for company in data['Company'].unique():
    window = 50
    no_of_std = 2
    df = data[data['Company']==company]['True Price']
    rolling_mean = df.rolling(window).mean()
    rolling_std = df.rolling(window).std()

    meanL.extend(rolling_mean)
    highL.extend(rolling_mean + (rolling_std * no_of_std))
    lowL.extend(rolling_mean - (rolling_std * no_of_std))
    

data['Rolling Mean'] = meanL
data['High'] = highL
data['Low'] = lowL

# Define Companies

In [46]:
# High variance stock
cList = ['SH600036', 'SH601318', 'SH600837', 'SH600406', 'SH601601', 'SH600585']

#cList = ['SH600000', 'SH600010', 'SH600015',  'SH600016', 'SH600018', 'SH600028',
#'SH600030', 'SH600036', 'SH600048', 'SH600050','SH600089','SH600104',"'SH600109","'SH601111",
#"'SH60150","SH600256",'SH600406','SH60518','SH600519','SH600583','SH600585','SH600637','SH600690',
#'SH600837','SH600887','SH600893','SH600958','SH600999','SH601006','SH601088','SH601166','SH601169',
#'SH601186','SH601288','SH601318','SH601328','SH601390','SH601398','SH601601','SH601628','SH601668','SH601688',
#'SH601766','SH601800','SH601818','SH601857','SH601901','SH601988','SH601989','SH601998']

# Prepare prediction function

In [47]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [48]:
def fit_model(decision_day, company):
    trainT = []
    testT = []
    for i in range(decision_day-100, decision_day):
        trainT.append(f'Day{i}')  
    
    X_train = data[(data['Time'].isin(trainT)) & (data['Company']==company)][["Price_rate5_change", "OBVlag", 'PROClag', "MA5lag", "day15Returnlag", 'Relative_10_LogVolume']]
    Y_train = data[(data['Time'].isin(trainT)) & (data['Company']==company)]['Next_1_Close']
    X_test = data[(data['Time']==f'Day{decision_day}') & (data['Company']==company)][["Price_rate5_change", "OBVlag", 'PROClag', "MA5lag", "day15Returnlag", 'Relative_10_LogVolume']]

    LR = LinearRegression(normalize=True).fit(X_train,Y_train)
    predict_LR_tst = LR.predict(X_test)

    return predict_LR_tst

# Prepare trading dataframe

In [49]:
trade = pd.DataFrame(index=range(504, 756))

trade['TradeDay']=trade.index+1
trade['StartC']=np.nan
trade['TransC']=0
trade['EndC']=np.nan

attribute=['StartU', 'PPrice', 'High', 'Low', 'APrice', 'BuyU', 'SellU', 'EndU' ]
for company in cList:
    for att in attribute:
        col = f'{company}_{att}'
        trade[col]=np.nan

trade.loc[504, 'StartC']=1000000
trade.loc[504, 'EndC']=1000000

for company in cList:
    trade.loc[504, f'{company}_StartU']=0
    trade.loc[504, f'{company}_EndU']=0
    for i in range(504, 756):
        #Bollinger Band
        com = data[(data['Company']==company)&(data['Time']==f'Day{i}')]
        trade.loc[i, f'{company}_High']=float(com.loc[:,'High'])
        trade.loc[i, f'{company}_Low']=float(com.loc[:,'Low'])
        trade.loc[i, f'{company}_APrice']=float(com.loc[:,'True Price'])

In [50]:
trade

,TradeDay,StartC,TransC,EndC,SH600036_StartU,SH600036_PPrice,SH600036_High,SH600036_Low,SH600036_APrice,SH600036_BuyU,SH600036_SellU,SH600036_EndU,SH601318_StartU,SH601318_PPrice,SH601318_High,SH601318_Low,SH601318_APrice,SH601318_BuyU,SH601318_SellU,SH601318_EndU,SH600837_StartU,SH600837_PPrice,SH600837_High,SH600837_Low,SH600837_APrice,SH600837_BuyU,SH600837_SellU,SH600837_EndU,SH600406_StartU,SH600406_PPrice,SH600406_High,SH600406_Low,SH600406_APrice,SH600406_BuyU,SH600406_SellU,SH600406_EndU,SH601601_StartU,SH601601_PPrice,SH601601_High,SH601601_Low,SH601601_APrice,SH601601_BuyU,SH601601_SellU,SH601601_EndU,SH600585_StartU,SH600585_PPrice,SH600585_High,SH600585_Low,SH600585_APrice,SH600585_BuyU,SH600585_SellU,SH600585_EndU
504,505,1000000.0,0,1000000.0,0.0,NaN,144.751397,131.791131,137.4271,NaN,NaN,0.0,0.0,NaN,158.939094,143.995858,148.6077,NaN,NaN,0.0,0.0,NaN,235.101976,175.160500,219.0830,NaN,NaN,0.0,0.0,NaN,496.999619,433.807801,497.4391,NaN,NaN,0.0,0.0,NaN,45.331521,36.841819,38.5864,NaN,NaN,0.0,0.0,NaN,157.722374,131.251158,140.0840,NaN,NaN,0.0
505,506,NaN,0,NaN,NaN,NaN,144.099422,131.991870,135.4195,NaN,NaN,NaN,NaN,NaN,158.277730,144.040218,145.6450,NaN,NaN,NaN,NaN,NaN,235.383261,175.387363,212.3969,NaN,NaN,NaN,NaN,NaN,498.942265,433.084239,498.2342,NaN,NaN,NaN,NaN,NaN,45.201379,36.717377,37.8640,NaN,NaN,NaN,NaN,NaN,156.421713,131.610315,138.0201,NaN,NaN,NaN
506,507,NaN,0,NaN,NaN,NaN,143.993248,131.880076,135.2283,NaN,NaN,NaN,NaN,NaN,158.380230,143.607582,144.0695,NaN,NaN,NaN,NaN,NaN,235.582292,176.070904,212.6197,NaN,NaN,NaN,NaN,NaN,500.548062,433.269966,495.3190,NaN,NaN,NaN,NaN,NaN,45.162864,36.586996,38.2376,NaN,NaN,NaN,NaN,NaN,155.965905,131.518083,139.0301,NaN,NaN,NaN
507,508,NaN,0,NaN,NaN,NaN,143.935881,131.652551,133.4118,NaN,NaN,NaN,NaN,NaN,158.495112,143.102372,143.0820,NaN,NaN,NaN,NaN,NaN,235.624872,176.786088,209.0538,NaN,NaN,NaN,NaN,NaN,501.926465,433.640683,493.9939,NaN,NaN,NaN,NaN,NaN,45.157013,36.412995,37.5900,NaN,NaN,NaN,NaN,NaN,155.645642,131.191942,135.4731,NaN,NaN,NaN
508,509,NaN,0,NaN,NaN,NaN,144.171733,130.875595,127.9626,NaN,NaN,NaN,NaN,NaN,158.721553,142.449859,141.3654,NaN,NaN,NaN,NaN,NaN,235.591658,177.550318,207.7165,NaN,NaN,NaN,NaN,NaN,502.661180,434.262860,485.7783,NaN,NaN,NaN,NaN,NaN,45.152822,36.245306,37.5651,NaN,NaN,NaN,NaN,NaN,155.088229,130.923783,133.9361,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,752,NaN,0,NaN,NaN,NaN,186.669885,173.792171,185.3978,NaN,NaN,NaN,NaN,NaN,221.949376,198.568312,206.1621,NaN,NaN,NaN,NaN,NaN,341.227640,302.023948,337.6927,NaN,NaN,NaN,NaN,NaN,656.361485,540.684239,591.6086,NaN,NaN,NaN,NaN,NaN,48.212920,42.956020,48.1896,NaN,NaN,NaN,NaN,NaN,227.341039,183.236897,221.3226,NaN,NaN,NaN
752,753,NaN,0,NaN,NaN,NaN,186.862892,173.844592,184.2688,NaN,NaN,NaN,NaN,NaN,221.676141,198.421043,205.7531,NaN,NaN,NaN,NaN,NaN,341.246896,302.013704,337.4674,NaN,NaN,NaN,NaN,NaN,656.450654,540.140610,572.9433,NaN,NaN,NaN,NaN,NaN,48.318603,42.907121,48.0873,NaN,NaN,NaN,NaN,NaN,228.149476,183.346556,221.1394,NaN,NaN,NaN
753,754,NaN,0,NaN,NaN,NaN,186.981739,173.837661,183.5816,NaN,NaN,NaN,NaN,NaN,221.350700,198.153736,202.7701,NaN,NaN,NaN,NaN,NaN,340.349142,302.469914,330.4838,NaN,NaN,NaN,NaN,NaN,656.089524,539.084260,573.2138,NaN,NaN,NaN,NaN,NaN,48.311082,42.910550,47.3328,NaN,NaN,NaN,NaN,NaN,229.464176,183.232156,227.6906,NaN,NaN,NaN
754,755,NaN,0,NaN,NaN,NaN,187.340977,173.747415,186.5759,NaN,NaN,NaN,NaN,NaN,221.028853,197.923251,203.1791,NaN,NaN,NaN,NaN,NaN,339.709098,302.803578,332.0607,NaN,NaN,NaN,NaN,NaN,656.216251,538.145997,568.3446,NaN,NaN,NaN,NaN,NaN,48.311082,42.910550,47.2177,NaN,NaN,NaN,NaN,NaN,230.861140,183.227904,229.8896,NaN,NaN,NaN


# Trade!

## Restrict continuous buying

In [9]:
cMark = {}
for company in cList:
    cMark[company]=0

for dDay in range(504, 756):
    if dDay > 504:
        trade.loc[dDay, 'StartC'] = trade.loc[dDay-1, 'EndC']
        trade.loc[dDay, 'EndC'] = trade.loc[dDay, 'StartC']
    for company in cList:
        trade.loc[dDay, f'{company}_PPrice']=fit_model(dDay, company)
        
        if dDay>504:
            trade.loc[dDay, f'{company}_StartU']=trade.loc[dDay-1, f'{company}_EndU']
        
        trade.loc[dDay, f'{company}_EndU']=trade.loc[dDay, f'{company}_StartU']
        trade.loc[dDay, f'{company}_BuyU']=0
        trade.loc[dDay, f'{company}_SellU']=0
        
        # Buying Signal: predicted price get close or exceed the lower band
        if trade.loc[dDay, f'{company}_PPrice']<trade.loc[dDay, f'{company}_Low']:
        #if trade.loc[dDay, f'{company}_PPrice']-trade.loc[dDay, f'{company}_Low'] <= trade.loc[dDay, f'{company}_PPrice']*0.01:
            if dDay<=700:
                # Money Constraint
                if (trade.loc[dDay, 'StartC'] > 0) and (cMark[company]<2) :
                    trade.loc[dDay, f'{company}_BuyU'] = (trade.loc[dDay, 'StartC']/2)//(trade.loc[dDay, f'{company}_APrice']*1.00065)
                    trade.loc[dDay, 'TransC'] += trade.loc[dDay, f'{company}_BuyU']*trade.loc[dDay, f'{company}_APrice']*0.00065
                    trade.loc[dDay, 'EndC'] -= trade.loc[dDay, f'{company}_BuyU']*trade.loc[dDay, f'{company}_APrice']*1.00065
                    trade.loc[dDay, f'{company}_EndU'] += trade.loc[dDay, f'{company}_BuyU']
                    cMark[company]+=1
        # Selling Signal: predicted price get close or exceed the upper band
        elif trade.loc[dDay, f'{company}_PPrice']>trade.loc[dDay, f'{company}_High']:
        #elif trade.loc[dDay, f'{company}_High']-trade.loc[dDay, f'{company}_PPrice'] <= trade.loc[dDay, f'{company}_PPrice']*0.01:
            if trade.loc[dDay, f'{company}_StartU'] > 0:
                trade.loc[dDay, f'{company}_SellU'] = trade.loc[dDay, f'{company}_StartU']
                trade.loc[dDay, 'TransC'] += trade.loc[dDay, f'{company}_SellU']*trade.loc[dDay, f'{company}_APrice']*0.00065
                trade.loc[dDay, 'EndC'] += trade.loc[dDay, f'{company}_SellU']*trade.loc[dDay, f'{company}_APrice']*(1-0.00065)
                trade.loc[dDay, f'{company}_EndU'] -= trade.loc[dDay, f'{company}_SellU']
                cMark[company]=0
        else:
            trade.loc[dDay, f'{company}_EndU']=trade.loc[dDay, f'{company}_StartU']
      
        if dDay==755:
            if trade.loc[dDay, f'{company}_StartU'] > 0:
                trade.loc[dDay, f'{company}_SellU'] = trade.loc[dDay, f'{company}_StartU']
                trade.loc[dDay, 'TransC'] += trade.loc[dDay, f'{company}_SellU']*trade.loc[dDay, f'{company}_APrice']*0.00065
                trade.loc[dDay, 'EndC'] += trade.loc[dDay, f'{company}_SellU']*trade.loc[dDay, f'{company}_APrice']*(1-0.00065)
                trade.loc[dDay, f'{company}_EndU'] -= trade.loc[dDay, f'{company}_SellU']
                

In [10]:
trade[trade['TransC']>0]

,TradeDay,StartC,TransC,EndC,SH600036_StartU,SH600036_PPrice,SH600036_High,SH600036_Low,SH600036_APrice,SH600036_BuyU,SH600036_SellU,SH600036_EndU,SH601318_StartU,SH601318_PPrice,SH601318_High,SH601318_Low,SH601318_APrice,SH601318_BuyU,SH601318_SellU,SH601318_EndU,SH600837_StartU,SH600837_PPrice,SH600837_High,SH600837_Low,SH600837_APrice,SH600837_BuyU,SH600837_SellU,SH600837_EndU,SH600406_StartU,SH600406_PPrice,SH600406_High,SH600406_Low,SH600406_APrice,SH600406_BuyU,SH600406_SellU,SH600406_EndU,SH601601_StartU,SH601601_PPrice,SH601601_High,SH601601_Low,SH601601_APrice,SH601601_BuyU,SH601601_SellU,SH601601_EndU,SH600585_StartU,SH600585_PPrice,SH600585_High,SH600585_Low,SH600585_APrice,SH600585_BuyU,SH600585_SellU,SH600585_EndU
508,509,1.000000e+06,324.730460,5.000899e+05,0.0,131.827417,144.171733,130.875595,127.9626,0.0,0.0,0.0,0.0,141.771186,158.721553,142.449859,141.3654,3534.0,0.0,3534.0,0.0,203.873959,235.591658,177.550318,207.7165,0.0,0.0,0.0,0.0,489.188885,502.661180,434.262860,485.7783,0.0,0.0,0.0,0.0,37.571947,45.152822,36.245306,37.5651,0.0,0.0,0.0,0.0,133.375716,155.088229,130.923783,133.9361,0.0,0.0,0.0
509,510,5.000899e+05,324.782270,1.001329e+02,0.0,128.986436,144.963537,129.598135,123.8517,2017.0,0.0,2017.0,3534.0,140.585391,159.136261,141.767095,138.7319,1801.0,0.0,5335.0,0.0,207.448793,234.915057,179.519575,208.8309,0.0,0.0,0.0,0.0,487.450041,502.395859,436.669529,485.5133,0.0,0.0,0.0,0.0,37.630771,45.200319,36.040373,36.9174,0.0,0.0,0.0,0.0,134.170218,155.100858,130.324470,132.3991,0.0,0.0,0.0
549,550,1.001329e+02,799.043896,1.228599e+06,2017.0,152.230524,149.983404,114.048716,155.4480,0.0,2017.0,0.0,5335.0,165.794854,160.026388,126.225004,171.6513,0.0,5335.0,0.0,0.0,304.115779,268.792310,176.657154,311.5748,0.0,0.0,0.0,0.0,526.975017,536.432740,474.844792,532.4215,0.0,0.0,0.0,0.0,44.645776,41.791868,32.982524,45.8354,0.0,0.0,0.0,0.0,154.189500,155.342406,119.520850,157.2981,0.0,0.0,0.0
598,599,1.228599e+06,797.817316,3.903182e+02,0.0,156.332749,175.994319,143.174985,154.9700,0.0,0.0,0.0,0.0,179.712061,208.250874,155.967850,180.2574,0.0,0.0,0.0,0.0,264.802128,347.959885,272.016051,268.5606,2285.0,0.0,2285.0,0.0,478.946974,615.023342,479.681902,487.1034,1260.0,0.0,1260.0,0.0,39.842034,47.809934,38.924002,39.9938,0.0,0.0,0.0,0.0,164.740278,194.104695,145.966557,165.6416,0.0,0.0,0.0
682,683,3.903182e+02,933.912932,1.436246e+06,0.0,173.209437,181.395106,165.785178,174.3535,0.0,0.0,0.0,0.0,213.061366,218.354732,200.579732,214.6539,0.0,0.0,0.0,2285.0,330.521727,328.942774,284.060522,337.9179,0.0,2285.0,0.0,1260.0,529.083065,518.829546,461.962774,527.4974,0.0,1260.0,0.0,0.0,48.570239,51.078176,44.176684,49.2767,0.0,0.0,0.0,0.0,196.294696,197.588851,171.643777,195.1634,0.0,0.0,0.0
694,695,1.436246e+06,466.456074,7.181543e+05,0.0,168.062770,181.417163,165.216397,167.2851,0.0,0.0,0.0,0.0,212.046696,220.725594,199.940682,213.9803,0.0,0.0,0.0,0.0,339.215459,359.548947,271.661877,343.7752,0.0,0.0,0.0,0.0,570.209376,560.210240,444.158224,581.5997,0.0,0.0,0.0,0.0,45.941600,50.601535,45.968901,45.3504,15824.0,0.0,15824.0,0.0,188.539693,198.154646,171.495798,188.9328,0.0,0.0,0.0
695,696,7.181543e+05,233.248441,3.590773e+05,0.0,169.538166,181.431561,165.013507,168.6104,0.0,0.0,0.0,0.0,212.917126,220.853391,200.060077,213.5714,0.0,0.0,0.0,0.0,337.339986,360.602428,271.716284,339.4949,0.0,0.0,0.0,0.0,579.442180,565.005676,442.795728,572.1318,0.0,0.0,0.0,15824.0,44.748065,50.680578,45.818806,45.5039,7886.0,0.0,23710.0,0.0,189.489054,198.270323,171.486405,190.0323,0.0,0.0,0.0
748,749,3.590773e+05,735.381299,1.489698e+06,0.0,182.843769,186.441155,172.418733,182.9926,0.0,0.0,0.0,0.0,207.479892,222.171796,199.006956,206.0177,0.0,0.0,0.0,0.0,332.960331,338.067522,303.210622,335.8904,0.0,0.0,0.0,0.0,570.034017,656.976181,538.457295,569.4266,0.0,0.0,0.0,23710.0,47.825630,47.724060,43.165056,47.7164,0.0,23710.0,0.0,0.0,225.139550,224.763295,182.305369,224.1172,0.0,0.0,0.0


## Not restrict continuous buying

In [51]:
cMark = {}
for company in cList:
    cMark[company]=0

for dDay in range(504, 756):
    if dDay > 504:
        trade.loc[dDay, 'StartC'] = trade.loc[dDay-1, 'EndC']
        trade.loc[dDay, 'EndC'] = trade.loc[dDay, 'StartC']
    for company in cList:
        trade.loc[dDay, f'{company}_PPrice']=fit_model(dDay, company)
        
        if dDay>504:
            trade.loc[dDay, f'{company}_StartU']=trade.loc[dDay-1, f'{company}_EndU']
        
        trade.loc[dDay, f'{company}_EndU']=trade.loc[dDay, f'{company}_StartU']
        trade.loc[dDay, f'{company}_BuyU']=0
        trade.loc[dDay, f'{company}_SellU']=0
        
        # Buying Signal: predicted price get close or exceed the lower band
        if trade.loc[dDay, f'{company}_PPrice']<trade.loc[dDay, f'{company}_Low']:
        #if trade.loc[dDay, f'{company}_PPrice']-trade.loc[dDay, f'{company}_Low'] <= trade.loc[dDay, f'{company}_PPrice']*0.01:
            if dDay<=700:
                # Money Constraint
                if trade.loc[dDay, 'StartC'] > 0:
                    trade.loc[dDay, f'{company}_BuyU'] = (trade.loc[dDay, 'StartC']/2)//(trade.loc[dDay, f'{company}_APrice']*1.00065)
                    trade.loc[dDay, 'TransC'] += trade.loc[dDay, f'{company}_BuyU']*trade.loc[dDay, f'{company}_APrice']*0.00065
                    trade.loc[dDay, 'EndC'] -= trade.loc[dDay, f'{company}_BuyU']*trade.loc[dDay, f'{company}_APrice']*1.00065
                    trade.loc[dDay, f'{company}_EndU'] += trade.loc[dDay, f'{company}_BuyU']
                    cMark[company]+=1
        # Selling Signal: predicted price get close or exceed the upper band
        elif trade.loc[dDay, f'{company}_PPrice']>trade.loc[dDay, f'{company}_High']:        
        #elif trade.loc[dDay, f'{company}_High']-trade.loc[dDay, f'{company}_PPrice'] <= trade.loc[dDay, f'{company}_PPrice']*0.01:
            if trade.loc[dDay, f'{company}_StartU'] > 0:
                trade.loc[dDay, f'{company}_SellU'] = trade.loc[dDay, f'{company}_StartU']
                trade.loc[dDay, 'TransC'] += trade.loc[dDay, f'{company}_SellU']*trade.loc[dDay, f'{company}_APrice']*0.00065
                trade.loc[dDay, 'EndC'] += trade.loc[dDay, f'{company}_SellU']*trade.loc[dDay, f'{company}_APrice']*(1-0.00065)
                trade.loc[dDay, f'{company}_EndU'] -= trade.loc[dDay, f'{company}_SellU']
                cMark[company]=0
        else:
            trade.loc[dDay, f'{company}_EndU']=trade.loc[dDay, f'{company}_StartU']
      
        if dDay==755:
            if trade.loc[dDay, f'{company}_StartU'] > 0:
                trade.loc[dDay, f'{company}_SellU'] = trade.loc[dDay, f'{company}_StartU']
                trade.loc[dDay, 'TransC'] += trade.loc[dDay, f'{company}_SellU']*trade.loc[dDay, f'{company}_APrice']*0.00065
                trade.loc[dDay, 'EndC'] += trade.loc[dDay, f'{company}_SellU']*trade.loc[dDay, f'{company}_APrice']*(1-0.00065)
                trade.loc[dDay, f'{company}_EndU'] -= trade.loc[dDay, f'{company}_SellU']
                

In [52]:
trade[trade['TransC']>0]

,TradeDay,StartC,TransC,EndC,SH600036_StartU,SH600036_PPrice,SH600036_High,SH600036_Low,SH600036_APrice,SH600036_BuyU,SH600036_SellU,SH600036_EndU,SH601318_StartU,SH601318_PPrice,SH601318_High,SH601318_Low,SH601318_APrice,SH601318_BuyU,SH601318_SellU,SH601318_EndU,SH600837_StartU,SH600837_PPrice,SH600837_High,SH600837_Low,SH600837_APrice,SH600837_BuyU,SH600837_SellU,SH600837_EndU,SH600406_StartU,SH600406_PPrice,SH600406_High,SH600406_Low,SH600406_APrice,SH600406_BuyU,SH600406_SellU,SH600406_EndU,SH601601_StartU,SH601601_PPrice,SH601601_High,SH601601_Low,SH601601_APrice,SH601601_BuyU,SH601601_SellU,SH601601_EndU,SH600585_StartU,SH600585_PPrice,SH600585_High,SH600585_Low,SH600585_APrice,SH600585_BuyU,SH600585_SellU,SH600585_EndU
508,509,1.000000e+06,324.730460,5.000899e+05,0.0,131.827417,144.171733,130.875595,127.9626,0.0,0.0,0.0,0.0,141.771186,158.721553,142.449859,141.3654,3534.0,0.0,3534.0,0.0,203.873959,235.591658,177.550318,207.7165,0.0,0.0,0.0,0.0,489.188885,502.661180,434.262860,485.7783,0.0,0.0,0.0,0.0,37.571947,45.152822,36.245306,37.5651,0.0,0.0,0.0,0.0,133.375716,155.088229,130.923783,133.9361,0.0,0.0,0.0
509,510,5.000899e+05,324.782270,1.001329e+02,0.0,128.986436,144.963537,129.598135,123.8517,2017.0,0.0,2017.0,3534.0,140.585391,159.136261,141.767095,138.7319,1801.0,0.0,5335.0,0.0,207.448793,234.915057,179.519575,208.8309,0.0,0.0,0.0,0.0,487.450041,502.395859,436.669529,485.5133,0.0,0.0,0.0,0.0,37.630771,45.200319,36.040373,36.9174,0.0,0.0,0.0,0.0,134.170218,155.100858,130.324470,132.3991,0.0,0.0,0.0
549,550,1.001329e+02,799.043896,1.228599e+06,2017.0,152.230524,149.983404,114.048716,155.4480,0.0,2017.0,0.0,5335.0,165.794854,160.026388,126.225004,171.6513,0.0,5335.0,0.0,0.0,304.115779,268.792310,176.657154,311.5748,0.0,0.0,0.0,0.0,526.975017,536.432740,474.844792,532.4215,0.0,0.0,0.0,0.0,44.645776,41.791868,32.982524,45.8354,0.0,0.0,0.0,0.0,154.189500,155.342406,119.520850,157.2981,0.0,0.0,0.0
598,599,1.228599e+06,797.817316,3.903182e+02,0.0,156.332749,175.994319,143.174985,154.9700,0.0,0.0,0.0,0.0,179.712061,208.250874,155.967850,180.2574,0.0,0.0,0.0,0.0,264.802128,347.959885,272.016051,268.5606,2285.0,0.0,2285.0,0.0,478.946974,615.023342,479.681902,487.1034,1260.0,0.0,1260.0,0.0,39.842034,47.809934,38.924002,39.9938,0.0,0.0,0.0,0.0,164.740278,194.104695,145.966557,165.6416,0.0,0.0,0.0
682,683,3.903182e+02,933.912932,1.436246e+06,0.0,173.209437,181.395106,165.785178,174.3535,0.0,0.0,0.0,0.0,213.061366,218.354732,200.579732,214.6539,0.0,0.0,0.0,2285.0,330.521727,328.942774,284.060522,337.9179,0.0,2285.0,0.0,1260.0,529.083065,518.829546,461.962774,527.4974,0.0,1260.0,0.0,0.0,48.570239,51.078176,44.176684,49.2767,0.0,0.0,0.0,0.0,196.294696,197.588851,171.643777,195.1634,0.0,0.0,0.0
694,695,1.436246e+06,466.456074,7.181543e+05,0.0,168.062770,181.417163,165.216397,167.2851,0.0,0.0,0.0,0.0,212.046696,220.725594,199.940682,213.9803,0.0,0.0,0.0,0.0,339.215459,359.548947,271.661877,343.7752,0.0,0.0,0.0,0.0,570.209376,560.210240,444.158224,581.5997,0.0,0.0,0.0,0.0,45.941600,50.601535,45.968901,45.3504,15824.0,0.0,15824.0,0.0,188.539693,198.154646,171.495798,188.9328,0.0,0.0,0.0
695,696,7.181543e+05,233.248441,3.590773e+05,0.0,169.538166,181.431561,165.013507,168.6104,0.0,0.0,0.0,0.0,212.917126,220.853391,200.060077,213.5714,0.0,0.0,0.0,0.0,337.339986,360.602428,271.716284,339.4949,0.0,0.0,0.0,0.0,579.442180,565.005676,442.795728,572.1318,0.0,0.0,0.0,15824.0,44.748065,50.680578,45.818806,45.5039,7886.0,0.0,23710.0,0.0,189.489054,198.270323,171.486405,190.0323,0.0,0.0,0.0
696,697,3.590773e+05,116.599131,1.795775e+05,0.0,171.465237,181.432364,165.014668,172.8809,0.0,0.0,0.0,0.0,211.709493,220.934736,200.180732,212.5370,0.0,0.0,0.0,0.0,333.437732,361.166102,271.971466,330.7090,0.0,0.0,0.0,0.0,555.455372,568.075770,442.606290,557.7947,0.0,0.0,0.0,23710.0,45.402215,50.744764,45.679732,45.6446,3930.0,0.0,27640.0,0.0,189.951884,198.543779,171.438349,192.4604,0.0,0.0,0.0
748,749,1.795775e+05,857.272842,1.497601e+06,0.0,182.843769,186.

In [76]:
trade.tail()

,TradeDay,StartC,TransC,EndC,SH600036_StartU,SH600036_PPrice,SH600036_High,SH600036_Low,SH600036_APrice,SH600036_BuyU,SH600036_SellU,SH600036_EndU,SH601318_StartU,SH601318_PPrice,SH601318_High,SH601318_Low,SH601318_APrice,SH601318_BuyU,SH601318_SellU,SH601318_EndU,SH600837_StartU,SH600837_PPrice,SH600837_High,SH600837_Low,SH600837_APrice,SH600837_BuyU,SH600837_SellU,SH600837_EndU,SH600406_StartU,SH600406_PPrice,SH600406_High,SH600406_Low,SH600406_APrice,SH600406_BuyU,SH600406_SellU,SH600406_EndU,SH601601_StartU,SH601601_PPrice,SH601601_High,SH601601_Low,SH601601_APrice,SH601601_BuyU,SH601601_SellU,SH601601_EndU,SH600585_StartU,SH600585_PPrice,SH600585_High,SH600585_Low,SH600585_APrice,SH600585_BuyU,SH600585_SellU,SH600585_EndU,total_assest,pnl
751,752,1.497601e+06,0.0,1.497601e+06,0.0,183.563526,186.669885,173.792171,185.3978,0.0,0.0,0.0,0.0,204.655548,221.949376,198.568312,206.1621,0.0,0.0,0.0,0.0,335.502051,341.227640,302.023948,337.6927,0.0,0.0,0.0,0.0,592.199788,656.361485,540.684239,591.6086,0.0,0.0,0.0,0.0,47.631346,48.212920,42.956020,48.1896,0.0,0.0,0.0,0.0,221.566111,227.341039,183.236897,221.3226,0.0,0.0,0.0,1.497601e+06,0.0
752,753,1.497601e+06,0.0,1.497601e+06,0.0,184.075148,186.862892,173.844592,184.2688,0.0,0.0,0.0,0.0,207.244369,221.676141,198.421043,205.7531,0.0,0.0,0.0,0.0,335.793849,341.246896,302.013704,337.4674,0.0,0.0,0.0,0.0,583.237784,656.450654,540.140610,572.9433,0.0,0.0,0.0,0.0,47.674657,48.318603,42.907121,48.0873,0.0,0.0,0.0,0.0,221.675724,228.149476,183.346556,221.1394,0.0,0.0,0.0,1.497601e+06,0.0
753,754,1.497601e+06,0.0,1.497601e+06,0.0,183.050168,186.981739,173.837661,183.5816,0.0,0.0,0.0,0.0,203.673833,221.350700,198.153736,202.7701,0.0,0.0,0.0,0.0,328.931625,340.349142,302.469914,330.4838,0.0,0.0,0.0,0.0,569.570190,656.089524,539.084260,573.2138,0.0,0.0,0.0,0.0,47.126767,48.311082,42.910550,47.3328,0.0,0.0,0.0,0.0,227.778400,229.464176,183.232156,227.6906,0.0,0.0,0.0,1.497601e+06,0.0
754,755,1.497601e+06,0.0,1.497601e+06,0.0,183.613376,187.340977,173.747415,186.5759,0.0,0.0,0.0,0.0,203.180224,221.028853,197.923251,203.1791,0.0,0.0,0.0,0.0,329.606460,339.709098,302.803578,332.0607,0.0,0.0,0.0,0.0,575.472682,656.216251,538.145997,568.3446,0.0,0.0,0.0,0.0,46.655029,48.311082,42.910550,47.2177,0.0,0.0,0.0,0.0,230.096839,230.861140,183.227904,229.8896,0.0,0.0,0.0,1.497601e+06,0.0
755,756,1.497601e+06,0.0,1.497601e+06,0.0,184.309406,187.580035,173.700773,185.4960,0.0,0.0,0.0,0.0,202.747694,220.642391,197.669817,201.9041,0.0,0.0,0.0,0.0,330.192290,339.104388,303.110920,331.1596,0.0,0.0,0.0,0.0,575.626380,656.191968,538.040436,586.4689,0.0,0.0,0.0,0.0,46.547521,48.314314,42.909366,47.0642,0.0,0.0,0.0,0.0,234.213667,232.436495,183.158877,232.7300,0.0,0.0,0.0,1.497601e+06,0.0


In [54]:
for i in range (504,756):
    trade.loc[i,"total_assest"] = (trade.loc[i,"EndC"]+(trade.loc[i,"SH600036_EndU"]*trade.loc[i,"SH600036_APrice"])+
                          (trade.loc[i,"SH601318_EndU"]*trade.loc[i,"SH601318_APrice"])+
                          (trade.loc[i,"SH600837_EndU"]*trade.loc[i,"SH600837_APrice"])+
                          (trade.loc[i,"SH600406_EndU"]*trade.loc[i,"SH600406_APrice"])+
                          (trade.loc[i,"SH601601_EndU"]*trade.loc[i,"SH601601_APrice"])+
                          (trade.loc[i,"SH600585_EndU"]*trade.loc[i,"SH600585_APrice"]))

In [55]:
for i in range (505,756):
    trade.loc[i,"pnl"] = (trade.loc[i,"total_assest"]/trade.loc[i-1,"total_assest"])-1

In [65]:
trade[trade["pnl"]!=0]

,TradeDay,StartC,TransC,EndC,SH600036_StartU,SH600036_PPrice,SH600036_High,SH600036_Low,SH600036_APrice,SH600036_BuyU,SH600036_SellU,SH600036_EndU,SH601318_StartU,SH601318_PPrice,SH601318_High,SH601318_Low,SH601318_APrice,SH601318_BuyU,SH601318_SellU,SH601318_EndU,SH600837_StartU,SH600837_PPrice,SH600837_High,SH600837_Low,SH600837_APrice,SH600837_BuyU,SH600837_SellU,SH600837_EndU,SH600406_StartU,SH600406_PPrice,SH600406_High,SH600406_Low,SH600406_APrice,SH600406_BuyU,SH600406_SellU,SH600406_EndU,SH601601_StartU,SH601601_PPrice,SH601601_High,SH601601_Low,SH601601_APrice,SH601601_BuyU,SH601601_SellU,SH601601_EndU,SH600585_StartU,SH600585_PPrice,SH600585_High,SH600585_Low,SH600585_APrice,SH600585_BuyU,SH600585_SellU,SH600585_EndU,total_assest,pnl
504,505,1000000.00000,0.000000,1.000000e+06,0.0,136.817665,144.751397,131.791131,137.4271,0.0,0.0,0.0,0.0,147.485684,158.939094,143.995858,148.6077,0.0,0.0,0.0,0.0,216.086325,235.101976,175.160500,219.0830,0.0,0.0,0.0,0.0,497.346090,496.999619,433.807801,497.4391,0.0,0.0,0.0,0.0,38.752618,45.331521,36.841819,38.5864,0.0,0.0,0.0,0.0,141.376499,157.722374,131.251158,140.0840,0.0,0.0,0.0,1.000000e+06,NaN
508,509,1000000.00000,324.730460,5.000899e+05,0.0,131.827417,144.171733,130.875595,127.9626,0.0,0.0,0.0,0.0,141.771186,158.721553,142.449859,141.3654,3534.0,0.0,3534.0,0.0,203.873959,235.591658,177.550318,207.7165,0.0,0.0,0.0,0.0,489.188885,502.661180,434.262860,485.7783,0.0,0.0,0.0,0.0,37.571947,45.152822,36.245306,37.5651,0.0,0.0,0.0,0.0,133.375716,155.088229,130.923783,133.9361,0.0,0.0,0.0,9.996753e+05,-0.000325
509,510,500089.94594,324.782270,1.001329e+02,0.0,128.986436,144.963537,129.598135,123.8517,2017.0,0.0,2017.0,3534.0,140.585391,159.136261,141.767095,138.7319,1801.0,0.0,5335.0,0.0,207.448793,234.915057,179.519575,208.8309,0.0,0.0,0.0,0.0,487.450041,502.395859,436.669529,485.5133,0.0,0.0,0.0,0.0,37.630771,45.200319,36.040373,36.9174,0.0,0.0,0.0,0.0,134.170218,155.100858,130.324470,132.3991,0.0,0.0,0.0,9.900437e+05,-0.009635
510,511,100.13287,0.000000,1.001329e+02,2017.0,124.067304,145.797775,128.043065,121.3661,0.0,0.0,2017.0,5335.0,138.996236,159.653041,140.730187,137.1800,0.0,0.0,5335.0,0.0,201.898309,233.808749,181.811563,203.4820,0.0,0.0,0.0,0.0,484.988510,502.444296,438.296004,483.3931,0.0,0.0,0.0,0.0,36.532574,45.222052,35.777008,36.1950,0.0,0.0,0.0,0.0,129.612248,154.573725,129.839835,129.9400,0.0,0.0,0.0,9.767509e+05,-0.013427
511,512,100.13287,0.000000,1.001329e+02,2017.0,122.856066,146.475785,126.735999,121.6529,0.0,0.0,2017.0,5335.0,138.322844,160.125753,139.791903,136.9684,0.0,0.0,5335.0,0.0,201.043054,232.276731,184.529261,201.0304,0.0,0.0,0.0,0.0,481.859083,502.249084,440.272140,483.6582,0.0,0.0,0.0,0.0,36.629108,45.266396,35.517436,36.0953,0.0,0.0,0.0,0.0,130.300201,154.365733,129.239819,130.4230,0.0,0.0,0.0,9.762004e+05,-0.000564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,745,179577.46408,0.000000,1.795775e+05,0.0,178.662484,186.810535,170.466825,177.2986,0.0,0.0,0.0,0.0,202.294791,222.150281,200.181251,201.0862,0.0,0.0,0.0,0.0,319.389854,339.147528,302.905572,318.9945,0.0,0.0,0.0,0.0,566.565192,656.698489,539.037967,557.2536,0.0,0.0,0.0,27640.0,44.821904,47.457796,43.236928,44.6470,0.0,0.0,27640.0,0.0,222.004449,219.618681,182.192483,221.2310,0.0,0.0,0.0,1.413621e+06,0.002507
745,746,179577.46408,0.000000,1.795775e+05,0.0,178.603829,186.482215,171.197649,178.6730,0.0,0.0,0.0,0.0,202.398930,222.188615,199.686809,202.1687,0.0,0.0,0.0,0.0,316.961298,337.948210,303.302898,319.4451,0.0,0.0,0.0,0.0,560.857656,657.032283,538.163149,558.6062,0.0,0.0,0.0,27640.0,44.927740,47.451059,43.229853,45.1586,0.0,0.0,27640.0,0.0,223.537500,221.286182,181.936022,225.3083,0.0,0.0,0.0,1.427761e+06,0.010003
746,747,179577.46408,0.000000,1.795775e+05,0.0,178.544642,186.402466,171.509082,178.6730,0.0,0.0,0.0,0.

In [67]:
trade["pnl"].sum()/250

0.0017049023191088874

In [70]:
np.std(trade["pnl"])

0.013347202256950599

In [60]:
## Metric 2: Sharpe Ratio
Sharpe_ratio = np.sqrt(252) * np.mean(trade["pnl"]) / np.std(trade["pnl"])
Sharpe_ratio

2.019648621681179

In [72]:
## Metric 3: number of days make profit 
np.sum(trade["pnl"] > 0)

82